In [1]:
%cd D:/uitds

D:\uitds


In [2]:
from collections import Counter
import json
from sklearn.metrics import f1_score

In [3]:
LOGNAME = 'merge_approach/log_merge_and_submit.txt'
def printlog(logname, log):
    with open(logname, 'a') as f:
        f.write(log + '\n')
# printlog(LOGNAME, 'start')

In [4]:
name_best1 = "v5_epoch7"
name_not = "v1_epoch2"
name_best2 = "v1_epoch5"
name_best3 = "v1_1_epoch7"

yes_no_file = f"result_ova/Vintern_train_yes_no_ova_{name_not}.json"
best_result_file = f"result_test_val/train_vintern_{name_best1}.json"
best_result_2_file = f"result_test_val/train_vintern_{name_best2}.json"
best_result_3_file = f"result_test_val/train_vintern_{name_best3}.json"

In [5]:
import json
with open(yes_no_file, 'r') as f:
    yes_no = json.load(f)

In [6]:
from collections import Counter
Counter(yes_no["results"].values())

Counter({'Yes': 827, 'No': 792})

In [7]:
printlog(LOGNAME,f"best_{name_best1}_best2_{name_best2}_not_{name_not}")

In [8]:
printlog(LOGNAME,f"best_{name_best1}_best2_{name_best2}_best3_{name_best3}_not_{name_not}_v2")

In [9]:
with open(best_result_file) as best_result, open(yes_no_file) as yes_no, open(best_result_2_file) as best_result_2, open(best_result_3_file) as best_result_3:
    yes_no = json.load(yes_no)
    best_result = json.load(best_result)
    best_result_2 = json.load(best_result_2)
    best_result_3 = json.load(best_result_3)

In [10]:
# read txt file: data\train_val_group_split.txt
TXT_PATH = "data/train_val_group_split.txt"
JSON_PATH = "data/json/vimmsd-train.json"

def read_train_test_file(name_file):
    with open(name_file, 'r') as f:
        lines = f.readlines()
        num_train, num_val = int(lines[0].split()[0]), int(lines[0].split()[1])
        train_idx = list(map(int, lines[1].split()))
        val_idx = list(map(int, lines[2].split()))
    return num_train, num_val, train_idx, val_idx

num_train, num_val, train_idx, val_idx = read_train_test_file(TXT_PATH)

json_file = open(JSON_PATH, 'r')
json_data = json.load(json_file)
labels = [json_data[str(i)]["label"] for i in val_idx]


In [11]:
import pandas as pd

df = pd.DataFrame(
    {'yes_no':yes_no["results"].values(), 
     'best_result': best_result["results"].values(), 
     'best_result_2': best_result_2["results"].values(),
     'best_result_3': best_result_3["results"].values(),
    'label': labels}
)

In [12]:
def old_transform_submit(best_result, yes_no):
    transformed_labels = []
    for i in range(len(best_result)):
        transformed_labels.append(best_result[i])
        if best_result[i] == "multi-sarcasm" and yes_no[i] == "No":
            transformed_labels[i] = "not-sarcasm"
        elif best_result[i] == "not-sarcasm" and yes_no[i] == "Yes":
            transformed_labels[i] = "multi-sarcasm"
    return transformed_labels

In [13]:
def transform_submit(best_result, best_result_2, yes_no):
    transformed_labels = []
    for i in range(len(best_result)):
        transformed_labels.append(best_result[i])
        if best_result[i] == "not-sarcasm" and best_result_2[i] == "not-sarcasm" and yes_no[i] == "Yes":
            transformed_labels[i] = "multi-sarcasm"
        elif best_result[i] == "not-sarcasm" and best_result_2[i] == "multi-sarcasm" and yes_no[i] == "Yes":
            transformed_labels[i] = "multi-sarcasm"
        elif best_result[i] == "multi-sarcasm" and best_result_2[i] == "multi-sarcasm" and yes_no[i] == "No":
            transformed_labels[i] = "not-sarcasm"
        elif best_result[i] == "not-sarcasm" and best_result_2[i] == "image-sarcasm" and yes_no[i] == "Yes":
            transformed_labels[i] = "image-sarcasm"
        elif best_result[i] == "multi-sarcasm" and best_result_2[i] == "image-sarcasm" and yes_no[i] == "Yes":
            transformed_labels[i] = "image-sarcasm"
    return transformed_labels

In [14]:
def transform_submit_2(best_result, best_result_2, best_result_3, yes_no):
    transformed_labels = []
    for i in range(len(best_result)):
        label1 = best_result[i]
        label2 = best_result_2[i]
        label3 = best_result_3[i]
        yes_no_label = yes_no[i]

        # Initialize the transformed label with the first result as default
        transformed_label = label1

        # Conditions for 'multi-sarcasm'
        if (
            # not-sarcasm not-sarcasm multi-sarcasm No: 6
            (label1 == 'not-sarcasm' and label2 == 'not-sarcasm' and label3 == 'multi-sarcasm' and yes_no_label == 'No') or
            # not-sarcasm multi-sarcasm not-sarcasm Yes: 18
            (label1 == 'not-sarcasm' and label2 == 'multi-sarcasm' and label3 == 'not-sarcasm' and yes_no_label == 'Yes') or
            # not-sarcasm image-sarcasm multi-sarcasm Yes: 2
            (label1 == 'not-sarcasm' and label2 == 'image-sarcasm' and label3 == 'multi-sarcasm' and yes_no_label == 'Yes') or
            # not-sarcasm not-sarcasm image-sarcasm Yes: 2
            (label1 == 'not-sarcasm' and label2 == 'not-sarcasm' and label3 == 'image-sarcasm' and yes_no_label == 'Yes') or
            # not-sarcasm not-sarcasm multi-sarcasm Yes: 3
            (label1 == 'not-sarcasm' and label2 == 'not-sarcasm' and label3 == 'multi-sarcasm' and yes_no_label == 'Yes') 
        ):
            transformed_label = 'multi-sarcasm'

        # Conditions for 'not-sarcasm'
        elif (
            # multi-sarcasm multi-sarcasm multi-sarcasm No: 19
            (label1 == 'multi-sarcasm' and label2 == 'multi-sarcasm' and label3 == 'multi-sarcasm' and yes_no_label == 'No') or
            # multi-sarcasm not-sarcasm not-sarcasm No: 17
            (label1 == 'multi-sarcasm' and label2 == 'not-sarcasm' and label3 == 'not-sarcasm' and yes_no_label == 'No') or
            # image-sarcasm multi-sarcasm multi-sarcasm Yes: 2
            (label1 == 'image-sarcasm' and label2 == 'multi-sarcasm' and label3 == 'multi-sarcasm' and yes_no_label == 'Yes')
        ):
            transformed_label = 'not-sarcasm'

        # Conditions for 'image-sarcasm'
        elif (
            # image-sarcasm image-sarcasm image-sarcasm Yes: 6
            (label1 == 'image-sarcasm' and label2 == 'image-sarcasm' and label3 == 'image-sarcasm' and yes_no_label == 'Yes') or
            # not-sarcasm image-sarcasm image-sarcasm Yes: 2
            (label1 == 'not-sarcasm' and label2 == 'image-sarcasm' and label3 == 'image-sarcasm' and yes_no_label == 'Yes')
        ):
            transformed_label = 'image-sarcasm'

        # Append the transformed label
        transformed_labels.append(transformed_label)

    return transformed_labels


In [15]:
printlog(LOGNAME,"train")
printlog(LOGNAME, f'f1: transform {f1_score(df["label"], transform_submit(df["best_result"], df["best_result_2"],df["yes_no"]), average="macro")}')
printlog(LOGNAME, f'f1: transform 2 {f1_score(df["label"], transform_submit_2(df["best_result"], df["best_result_2"], df["best_result_3"],df["yes_no"]), average="macro")}')
printlog(LOGNAME, f'f1: old transform {f1_score(df["label"], old_transform_submit(df["best_result"], df["yes_no"]), average="macro")}')
printlog(LOGNAME, f'f1: best1 {f1_score(df["label"], df["best_result"], average="macro")}')

In [16]:
printlog(LOGNAME, f'transform {Counter(transform_submit(df["best_result"], df["best_result_2"],df["yes_no"]))}')
printlog(LOGNAME, f'transform 2 {Counter(transform_submit_2(df["best_result"], df["best_result_2"], df["best_result_3"],df["yes_no"]))}')
printlog(LOGNAME, f'old transform {Counter(old_transform_submit(df["best_result"], df["yes_no"]))}')
printlog(LOGNAME, f'best1 {Counter(df["best_result"])}')

In [17]:
results = {}
results["results"] = {}
results["phase"] = "dev"
merge_val = old_transform_submit(df["best_result"], df["yes_no"])
for i in range(len(merge_val)):
    results["results"][str(i)] = merge_val[i]
with open(f"merge_val/best_not/val_best_{name_best1}_not_{name_not}_results.json", "w") as out:
    json.dump(results, out)

In [18]:
results = {}
results["results"] = {}
results["phase"] = "dev"
merge_val = transform_submit(df["best_result"], df["best_result_2"],df["yes_no"])
for i in range(len(merge_val)):
    results["results"][str(i)] = merge_val[i]
with open(f"merge_val/best1_best2_not/val_best_{name_best1}_best2_{name_best2}_not_{name_not}_results.json", "w") as out:
    json.dump(results, out)

In [19]:
results = {}
results["results"] = {}
results["phase"] = "dev"
merge_val = transform_submit_2(df["best_result"], df["best_result_2"],df["best_result_3"], df["yes_no"])
for i in range(len(merge_val)):
    results["results"][str(i)] = merge_val[i]
with open(f"merge_val/best1_best2_best3_not_v2/val_best_{name_best1}_best2_{name_best2}_best3_{name_best3}_not_{name_not}_v2_results.json", "w") as out:
    json.dump(results, out)

In [20]:
yes_no_file = f"result_ova/Vintern_test_yes_no_ova_{name_not}.json"
best_result_file = f"result_test_val/test_vintern_{name_best1}.json"
best_result_2_file = f"result_test_val/test_vintern_{name_best2}.json"
best_result_3_file = f"result_test_val/test_vintern_{name_best3}.json"

In [21]:
with open(best_result_file) as best_result, open(yes_no_file) as yes_no, open(best_result_2_file) as best_result_2, open(best_result_3_file) as best_result_3:
    yes_no = json.load(yes_no)
    best_result = json.load(best_result)
    best_result_2 = json.load(best_result_2)
    best_result_3 = json.load(best_result_3)

In [22]:
df = pd.DataFrame({'yes_no':yes_no["results"].values(), 'best_result': best_result["results"].values(),'best_result_2': best_result_2["results"].values(), 'best_result_3': best_result_3["results"].values()})

In [23]:
printlog(LOGNAME,"test")
printlog(LOGNAME, f'transform {Counter(transform_submit(df["best_result"], df["best_result_2"],df["yes_no"]))}')
printlog(LOGNAME, f'transform 2 {Counter(transform_submit_2(df["best_result"], df["best_result_2"], df["best_result_3"],df["yes_no"]))}')
printlog(LOGNAME, f'old transform {Counter(old_transform_submit(df["best_result"], df["yes_no"]))}')
printlog(LOGNAME, f'best1 {Counter(df["best_result"])}')

In [24]:
printlog(LOGNAME, "\n")

In [25]:
results = {}
results["results"] = {}
results["phase"] = "dev"
merge_val = transform_submit(df["best_result"], df["best_result_2"],df["yes_no"])
for i in range(len(merge_val)):
    results["results"][str(i)] = merge_val[i]

In [26]:
with open(f"merge_test/best_{name_best1}_best2_{name_best2}_not_{name_not}_results.json", "w") as out:
    json.dump(results, out)

In [27]:
import os
import shutil
import zipfile
def create_dir(dir_name):
    try:
        os.mkdir(dir_name)
    except FileExistsError:
        print("DIRECTORY ALREADY EXISTS!!!")
    except Exception as e:
        print(e)
    else:
        print("Directory created successfully")

In [28]:
def copy_rename_zip(name, new_file_name, new_file_ext):
    file_name = f"merge_test/{name}_results.json"
    dir_name = f"submissions/{name}"
    create_dir(dir_name)

    new_file_path = os.path.join(dir_name, new_file_name)
    full_file_path = new_file_path + new_file_ext
    
    try:
        shutil.copy(file_name, full_file_path)
    except FileNotFoundError:
        print("FILE NOT FOUND!!!")
        os.rmdir(dir_name)
        return
        
    with zipfile.ZipFile(new_file_path + '.zip', 'w') as myzip:
        myzip.write(full_file_path, arcname=new_file_name + new_file_ext)
    return

In [29]:
copy_rename_zip(f"best_{name_best1}_best2_{name_best2}_not_{name_not}", "results", ".json")

DIRECTORY ALREADY EXISTS!!!


In [30]:
f"best_{name_best1}_best2_{name_best2}_not_{name_not}"

'best_v5_epoch7_best2_v1_epoch5_not_v1_epoch2'

In [31]:
results = {}
results["results"] = {}
results["phase"] = "dev"
merge_val = old_transform_submit(df["best_result"], df["yes_no"])
for i in range(len(merge_val)):
    results["results"][str(i)] = merge_val[i]

In [32]:
with open(f"merge_test/best_{name_best1}_not_{name_not}_results.json", "w") as out:
    json.dump(results, out)

In [33]:
copy_rename_zip(f"best_{name_best1}_not_{name_not}", "results", ".json")

DIRECTORY ALREADY EXISTS!!!


In [34]:
results = {}
results["results"] = {}
results["phase"] = "dev"
merge_val = transform_submit_2(df["best_result"], df["best_result_2"],df["best_result_3"],df["yes_no"])
for i in range(len(merge_val)):
    results["results"][str(i)] = merge_val[i]

In [35]:
with open(f"merge_test/best_{name_best1}_best2_{name_best2}_best3_{name_best3}_not_{name_not}_v2_results.json", "w") as out:
    json.dump(results, out)

In [36]:
f"best_{name_best1}_best2_{name_best2}_best3_{name_best3}_not_{name_not}_v2"

'best_v5_epoch7_best2_v1_epoch5_best3_v1_1_epoch7_not_v1_epoch2_v2'

In [37]:
copy_rename_zip(f"best_{name_best1}_best2_{name_best2}_best3_{name_best3}_not_{name_not}_v2", "results", ".json")

DIRECTORY ALREADY EXISTS!!!
